In [ ]:
from flask import Flask, render_template, request, jsonify

import serial.tools.list_ports
import struct
import threading
import time
import logging
import os

app = Flask(__name__, template_folder='HTML')

log = logging.getLogger('werkzeug')
log.setLevel(logging.ERROR)

def close_uart():
    global serial_connection
    if serial_connection.is_open:
        serial_connection.close()
        print("Cổng UART đã được đóng.")

# Biến lưu trữ dữ liệu nhận từ UART
update_data = []
received_data = []
serial_connection = None
default_baudrate = 115200
def hex_to_str(hex_string: str) -> str:
    try:
        # Chuyển chuỗi hex thành chuỗi bytes
        bytes_data = bytes.fromhex(hex_string)
        # Giải mã bytes thành chuỗi str
        return bytes_data.decode('ascii')
    except ValueError:
        return "Invalid hex string"
    except UnicodeDecodeError:
        return "Non-ASCII characters in input"

# Hàm tính CRC-16 Modbus
def calculate_crc(data: bytes) -> bytes:
    crc = 0xFFFF
    for b in data:
        crc ^= b
        for _ in range(8):
            if crc & 0x0001:
                crc = (crc >> 1) ^ 0xA001
            else:
                crc >>= 1
    return struct.pack('<H', crc)  # Trả về CRC dưới dạng 2 byte (little-endian)
def parse_data(input_str):
    # Kiểm tra và tìm vị trí kết thúc "0D0A"
    end_index = input_str.find("0D0A")
    
    # Nếu không tìm thấy "0D0A", báo lỗi hoặc xử lý thêm
    if end_index == -1:
        return "Invalid Input: Missing CRLF"
    
    # Lấy phần dữ liệu từ đầu đến trước "0D0A"
    input_str = input_str[:end_index]
    
    # Start: ký tự đầu tiên trong chuỗi
    start = input_str[:2]  # 2 ký tự đầu tiên cho Start
    
    # Addr: 2 ký tự tiếp theo
    addr = input_str[2:4]
    
    # Code: 2 ký tự tiếp theo sau Addr
    code = input_str[4:6]

    int_code = int(code, 16)

    if int_code > 127:
        code = f"{(int_code-127):02X}"

    
    # Phần Data: Các cặp ký tự sau Code cho đến phần CRC
    data_index = 6
    data_array = []
    
    while data_index < len(input_str) - 4:
        data_array.append(input_str[data_index:data_index + 2])
        data_index += 2
    data = "".join(data_array)
    
    # Tính phần CRC (phần cuối của chuỗi sau Data)
    crc = input_str[-4:]  # Giả sử CRC là 4 ký tự cuối cùng
    
    # Stop: Kết thúc "CRLF"
    stop = "CRLF"
    frame = bytearray()
    frame.append(0x3A)  # Thêm mã hàm
    frame.append(int(addr, 16))  # Thêm mã hàm
    frame.append(int_code)  # Thêm mã hàm
    frame.extend(hex_to_str(data).encode('ascii'))  # Thêm dữ liệu người dùng
    cc = calculate_crc(frame)  # Tính CRC
    # Tạo chuỗi kết quả
    result = f"{start} {addr} {code} {data} {crc} {stop}"
    val = ""
    try:
        keyval = data_codes[int(code, 16)]
        val = hex_to_str(data)
    except:
        keyval = data_codes[5]
        val = "Undefine Code"
    Arr = {
        'start':start,
        'addr':addr.upper(),
        'code':code.upper(),
        'data':data.upper(),
        'crc':crc.upper(),
        'stop':stop, 
        'key': keyval,
        'value': val,
        'update':int_code>127,
        'exac':crc.upper()== cc.hex().upper()
    }
    print(f"Caculate {hex_to_str(data)} {cc.hex().upper()} {Arr}")

    return Arr


# Gửi dữ liệu qua RS485
def send_rs485(address: int ,function_code: int, user_data: str):
    # Tạo frame RS485
    global serial_connection
    frame = bytearray()
    frame.append(0x3A)  # Thêm mã hàm
    frame.append(address)  # Thêm mã hàm
    frame.append(function_code)  # Thêm mã hàm
    frame.extend(user_data.encode('ascii'))  # Thêm dữ liệu người dùng
    crc = calculate_crc(frame)  # Tính CRC
    frame.extend(crc)  # Thêm CRC vào cuối frame
    frame.extend(b'\r\n')  # Thêm CRLF
    # Gửi frame qua UART
    # uart.write(":")
    serial_connection.write(frame.hex().encode('ascii'))
    # uart.write(frame.hex().encode('ascii'))
    print(f"Buffer: {frame.hex().upper()}")
    result = f"'3A {address:02X} {function_code:02X} {user_data.encode('ascii').hex()} {crc.hex()} CRLF'"
    Arr = {
        'start':"3A",
        'addr':f"{address:02X}",
        'code':f"{function_code:02X}",
        'data':user_data.encode('ascii').hex().upper(),
        'crc':crc.hex().upper(),
        'stop':"CRLF"
    }
    # return result.upper()
    return Arr
    # return frame.hex().upper()  # Trả về dữ liệu gửi dưới dạng hex

# Đọc dữ liệu từ UART
def read_from_uart():
    global received_data
    global serial_connection
    while True:
        try:
            if serial_connection and serial_connection.is_open and serial_connection.in_waiting > 0:
            # if uart.in_waiting > 0:
                # Đọc dữ liệu từ UART
                data = serial_connection.read_until(b'\r\n')#.decode('utf-8').strip()
                # data = uart.read_until(b'\r\n')#.decode('utf-8').strip()
                # uart.read_until()
                print(f"Data Read {data.decode()}")
                # Kiểm tra xem dữ liệu có hợp lệ không (không phải None và không rỗng)
                dat = parse_data(data.decode())
                # if():
                    
                print(f"Read: {dat}")
                if dat['update'] == False:
                    global frame_sent
                    received_data.append({'send':frame_sent,'receiver': dat})
                    # received_data.append(data)
                else:
                    update_data.append({'receiver': dat})
        except serial.SerialException as e:
            print(f"Error reading from UART: {e}")
            break  # Thoát vòng lặp nếu có lỗi với cổng serial
        except Exception as e:
            print(f"Unexpected error: {e}")
            break  # Thoát vòng lặp nếu có lỗi ngoài cổng serial


# Khởi động luồng đọc dữ liệu từ UART
# thread = threading.Thread(target=read_from_uart, daemon=True)
# thread.start()

# Lựa chọn mã hàm
function_codes = {
    1: "Đọc dữ liệu",
    2: "Gửi thông báo",
    3: "Kiểm tra trạng thái",
    4: "Chỉnh C",
    5: "Chỉnh D",
    6: "Chỉnh T",
}
data_codes = {
    1: "Nhiệt độ ",
    2: "Áp Suất ",
    3: "Bàn Phím ",
    4: "Đếm Xung ",
    5: "Phản hồi ",
}
def get_available_ports():
    # Sử dụng thư viện pyserial để liệt kê cổng
    ports = serial.tools.list_ports.comports()
    available_ports = [port.device for port in ports]
    
    if not available_ports:
        print("Không tìm thấy cổng COM nào khả dụng.")
    return available_ports


# Giao diện trang chủ
@app.route('/')
def index():
    ports = get_available_ports()
    # print(ports)
    return render_template("Index.html", ports=ports, default_baudrate=default_baudrate)

# API gửi dữ liệu
@app.route('/send', methods=['POST'])
def send():
    try:
        global frame_sent
        address = int(request.json.get('address'))
        function_code = int(request.json.get('function_code'))
        user_data = request.json.get('user_data')

        if function_code not in function_codes:
            return jsonify({'status': 'error', 'message': 'Mã hàm không hợp lệ!'})

        # Gửi dữ liệu qua UART và nhận frame đã gửi
        frame_sent = send_rs485(address, function_code, user_data)
        result = f"'3A {frame_sent['addr']} {frame_sent['code']} {frame_sent['data']} {frame_sent['crc']} CRLF'"
        print(f"Send: {result}")
        return jsonify({'status': 'success', 'message': 'Data sent successfully!', 'frame_sent': result})

    except Exception as e:
        return jsonify({'status': 'error', 'message': str(e)})

# API nhận dữ liệu
@app.route('/receive', methods=['GET'])
def receive():
    global received_data
    data = received_data.copy()
    # data.append(frame_sent)
    received_data = []  # Xóa dữ liệu sau khi gửi
    return jsonify(data)
# API nhận dữ liệu
@app.route('/update', methods=['GET'])
def update():
    global update_data
    data = update_data.copy()
    # data.append(frame_sent)
    update_data = []  # Xóa dữ liệu sau khi gửi
    return jsonify(data)
@app.route("/open_port", methods=["POST"])
def open_port():
    """Mở cổng serial."""
    global serial_connection
    data = request.json
    port = data.get("port")
    baudrate = data.get("baudrate", default_baudrate)

    try:
        # Đóng kết nối cũ nếu tồn tại
        if serial_connection and serial_connection.is_open:
            serial_connection.close()

        # Mở cổng serial
        serial_connection = serial.Serial(port, baudrate, timeout=0.2)
        thread = threading.Thread(target=read_from_uart, daemon=True)
        thread.start()
        return jsonify({"status": "success", "message": f"Open {port} Baudrate {baudrate}."})
    except Exception as e:
        return jsonify({"status": "error", "message": str(e)})

@app.route("/close_port", methods=["POST"])
def close_port():
    """Đóng cổng serial."""
    global serial_connection
    try:
        if serial_connection and serial_connection.is_open:
            serial_connection.close()
            return jsonify({"status": "success", "message": "Close Serial Port."})
        else:
            return jsonify({"status": "error", "message": "Cannot Open Serial Port."})
    except Exception as e:
        return jsonify({"status": "error", "message": str(e)})

# Chạy Flask
try:
    if __name__ == '__main__':
        print('Go to: http://127.0.0.1:5000/')
        app.run(debug=False, port=5000)
except:
    pass
finally:
    close_uart()


Data Read 3A6F8032362E304340120D0A
Caculate 26.0C 4012 {'start': '3A', 'addr': '6F', 'code': '01', 'data': '32362E3043', 'crc': '4012', 'stop': 'CRLF', 'key': 'Nhiệt độ ', 'value': '26.0C', 'update': True, 'exac': True}
Read: {'start': '3A', 'addr': '6F', 'code': '01', 'data': '32362E3043', 'crc': '4012', 'stop': 'CRLF', 'key': 'Nhiệt độ ', 'value': '26.0C', 'update': True, 'exac': True}
Data Read 3A6F8131303131323250615E450D0A
Caculate 101122Pa 5E45 {'start': '3A', 'addr': '6F', 'code': '02', 'data': '3130313132325061', 'crc': '5E45', 'stop': 'CRLF', 'key': 'Áp Suất ', 'value': '101122Pa', 'update': True, 'exac': True}
Read: {'start': '3A', 'addr': '6F', 'code': '02', 'data': '3130313132325061', 'crc': '5E45', 'stop': 'CRLF', 'key': 'Áp Suất ', 'value': '101122Pa', 'update': True, 'exac': True}
Data Read 3A6F8032362E304340120D0A
Caculate 26.0C 4012 {'start': '3A', 'addr': '6F', 'code': '01', 'data': '32362E3043', 'crc': '4012', 'stop': 'CRLF', 'key': 'Nhiệt độ ', 'value': '26.0C', 'up